In [ ]:
pip install -r requirements.txt

In [15]:
import weaviate
import os
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
import pickle

In [16]:
file_name = "yoochoose-clicks.dat"
pkl_file_name= "yoochoose_trigrams.pkl"

columns = ["session_id", "ts", "item_id", "category_id"]
column_use = ["session_id", "item_id"]

dtype_mapping = {
    "session_id": "UInt32",
    "ts": "str",
    "item_id": "UInt32",
    "category_id": "category"
}

In [18]:
# # Load environment variables from .env file
load_dotenv()

# # Get the file path from the environment variable
file_path = os.getenv("PATH_TO_ORIGINAL_DATA")
model_path = os.getenv('PATH_TO_MODEL')
print(file_path)
print(model_path)

C:\Users\Mohammed Zoghian\Documents\weaviate_embeddings\Dataset\
'C:\Users\Mohammed Zoghian\Documents\weaviate_embeddings\models\'


In [6]:
# Load the data
# Data Source: https://www.kaggle.com/datasets/chadgostopp/recsys-challenge-2015
data = pd.read_csv(file_path + file_name, names=columns, usecols= column_use, dtype=dtype_mapping)

print(data.head())

   session_id    item_id
0           1  214536502
1           1  214536500
2           1  214536506
3           1  214577561
4           2  214662742


In [12]:
#load vocabulary mapping from pickle file
with open(model_path + pkl_file_name, "rb") as f:
    vocab = pickle.load(f)

vocab_mapping = vocab.get("vocab_mapping", {})
print("Vocab Mapping:", vocab_mapping)

OSError: [Errno 22] Invalid argument: "'C:\\Users\\Mohammed Zoghian\\Documents\\weaviate_embeddings\\models\\'yoochoose_trigrams.pkl"